In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [25]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose #this is where we are importing the pre-built pose detection model from mediapipe


In [27]:
from IPython.display import HTML

# Define the HTML content
html_content = '<image src="https://i.imageur.com/3j8BPdc.png" style="height:300px">'

# Display the HTML content
display(HTML(html_content))


In [28]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [29]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

landmarks = ['class']
for val in range(1, 34): #because 33 points
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]


In [30]:
landmarks[1:]

['x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 'x33',
 'y3

In [31]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [32]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints, 0, action)  # Insert action at the beginning

        with open('coords.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print('Error occurred while exporting landmarks:', e)


In [9]:
#video feed
cap = cv2.VideoCapture('squats.mp4') #this number is the number of my web cam

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False #save a bunch of memory

        #make detections from the pose
        results = pose.process(image) #by processing it, we get our detections back
    
        #back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
       
        k = cv2.waitKey(1)
        if k == 117:
            export_landmark(results, '1')
        if k == 100:
            export_landmark(results, '2')
            
        cv2.imshow('Raw Webcam feed', image)
        
       # cv2.imshow('Mediapipe Feed', image)        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [10]:
cap.release()
cv2.destroyAllWindows()


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('coords.csv')

In [3]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,2.0,0.134881,0.397049,0.244450,0.999933,0.149391,0.379769,0.190907,0.999945,0.155251,...,0.402558,0.658958,0.292590,0.978475,-0.713149,0.986467,0.379712,0.940524,0.374028,0.719632
1,1.0,0.342728,0.128654,-0.009005,0.999922,0.367347,0.115402,0.003467,0.999926,0.377168,...,0.152534,0.446672,0.298999,0.961971,-0.900583,0.945800,0.383854,0.916545,0.084651,0.523301
2,1.0,0.339378,0.130847,-0.097485,0.999691,0.361264,0.116912,-0.096245,0.999679,0.370950,...,0.131494,0.459408,0.304292,0.980623,-0.747078,0.934796,0.392010,0.913562,0.071831,0.537464
3,1.0,0.338869,0.130923,-0.050493,0.999533,0.360098,0.116909,-0.066604,0.999496,0.369351,...,0.044729,0.466513,0.324522,0.956619,-0.683048,0.927368,0.392671,0.914389,-0.049897,0.546686
4,1.0,0.337715,0.132803,-0.068850,0.999459,0.358700,0.117711,-0.082483,0.999403,0.368147,...,-0.045246,0.467822,0.322610,0.958019,-0.732507,0.919448,0.390270,0.915377,-0.140995,0.547931


In [4]:
df.tail()


,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
55,2.0,0.187767,0.403471,0.367159,0.999639,0.205636,0.389896,0.305645,0.999824,0.210675,...,1.016456,0.446929,0.211564,0.994628,-0.972186,0.927909,0.443286,0.888615,1.064162,0.483483
56,1.0,0.277921,0.134553,0.291469,0.999907,0.303072,0.122394,0.284146,0.999942,0.309467,...,-0.090860,0.471862,0.305248,0.988466,-0.917308,0.865143,0.342527,0.903945,-0.177643,0.469619
57,1.0,0.307017,0.133174,0.021628,0.999935,0.335923,0.120957,-0.017172,0.999951,0.342328,...,0.162533,0.413991,0.314942,0.967729,-0.567960,0.853277,0.358825,0.919785,0.027955,0.421898
58,1.0,0.318101,0.131008,0.078367,0.999937,0.347548,0.119896,0.050084,0.999945,0.354037,...,0.074661,0.400096,0.312894,0.960464,-0.596535,0.857965,0.370635,0.919702,-0.051798,0.411522
59,1.0,0.321200,0.130668,0.081991,0.999943,0.350280,0.119937,0.053323,0.999947,0.356678,...,0.066998,0.397125,0.317246,0.956600,-0.585473,0.860786,0.375920,0.919366,-0.069747,0.406993


In [5]:
df[df['class'] == '1.0']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33


In [6]:
X = df.drop('class', axis=1)
y = df['class']


In [7]:
#train_test_split is a function from the sklearn.model_selection module that is used to
# split arrays or matrices into random train and test subsets.

#class is my target variable and I want to predict that

#X is the dataset with no class coulmn
#y extracts the targer variable class

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1234)

In [8]:
y_test

29    1.0
45    2.0
8     2.0
22    1.0
6     1.0
54    2.0
39    1.0
13    1.0
18    2.0
31    2.0
51    1.0
33    1.0
4     1.0
20    2.0
46    2.0
27    1.0
21    1.0
50    1.0
Name: class, dtype: float64

In [9]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
29,0.344602,0.141248,0.061751,0.999895,0.365353,0.125057,0.056393,0.999914,0.375429,0.124402,...,0.161402,0.558037,0.287479,0.951268,-0.875654,0.944361,0.377761,0.903523,0.125897,0.628971
45,0.109509,0.514019,0.207938,0.998616,0.111218,0.494612,0.150279,0.999507,0.114795,0.492811,...,1.119906,0.534957,0.189163,0.968756,-0.570077,0.943820,0.405401,0.893321,1.200961,0.522077
8,0.116053,0.401095,0.164450,0.999745,0.135661,0.387292,0.115094,0.999822,0.141870,0.387754,...,0.697075,0.610349,0.251919,0.978852,-0.657969,0.978404,0.379350,0.918628,0.726558,0.643255
22,0.390301,0.142653,0.217323,0.999609,0.413343,0.130849,0.221329,0.999665,0.417989,0.131228,...,-0.109194,0.466372,0.324234,0.950591,-0.872560,0.931121,0.400082,0.906735,-0.183673,0.588500
6,0.313949,0.143241,-0.028665,0.999346,0.328266,0.125524,-0.060125,0.999285,0.337055,0.123648,...,0.103544,0.467413,0.321157,0.955727,-0.752331,0.914653,0.386411,0.913522,0.036598,0.550628
54,0.187355,0.424089,0.177779,0.999666,0.206668,0.407784,0.129017,0.999817,0.212339,0.407892,...,1.073631,0.445609,0.235153,0.999482,-0.563273,0.923915,0.461286,0.888280,1.129841,0.475304
39,0.339174,0.142989,0.190577,0.999914,0.361667,0.124132,0.190933,0.999915,0.369487,0.123532,...,-0.220709,0.646724,0.310085,0.949220,-0.844575,0.956608,0.381217,0.918313,-0.333679,0.708883
13,0.299925,0.148300,0.255391,0.998495,0.315585,0.133819,0.218465,0.998982,0.319091,0.133515,...,0.013093,0.473499,0.313376,0.971788,-0.679634,0.930526,0.400110,0.907216,-0.043901,0.565754
18,0.158479,0.406725,0.004577,0.999859,0.179170,0.392727,-0.014446,0.999899,0.188235,0.392972,...,0.592709,0.549415,0.237346,0.979091,-0.711949,0.980138,0.356805,0.895148,0.603129,0.597349
31,0.142160,0.423136,0.160556,0.999976,0.161481,0.404632,0.141434,0.999980,0.170716,0.403883,...,0.714152,0.546981,0.239005,1.027022,-0.886542,0.973866,0.432330,0.898081,0.793980,0.545797


In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [11]:
pipelines = {
    #StandardScaler() before the Random Forest Classifier. 
    # The StandardScaler standardizes features by removing the mean and scaling to unit variance. 
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [12]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [13]:
fit_models = {} #create empty dictionary
for algorithm, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algorithm] = model #put the results of trained (fitted) models in the dictionary

In [14]:
fit_models['rf'].predict(X_test)

array([1., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1., 1.,
       1.])

In [15]:
fit_models['rf'].predict(X_test)

array([1., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1., 1.,
       1.])

In [16]:
fit_models['lr'].predict(X_test)

array([1., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1., 1.,
       1.])

We just used the pipeline to train different models on X_train and y_train and store the collected trainer models in fit_models

Next step:
Evaluate and serialize model

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #accuracy metrics
import pickle

In [18]:
yhat = model.predict(X_test)
yhat

array([1., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1., 1.,
       1.])

In [19]:
for algorithm, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algorithm, accuracy_score(y_test.values, yhat),
          precision_score(y_test.values, yhat, average="binary", pos_label=1),
          recall_score(y_test.values, yhat, average="binary", pos_label=2))
    

rf 1.0 1.0 1.0
lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [20]:
yhat = fit_models['rf'].predict(X_test)

In [21]:
yhat[:10]

#slicing yhat to display the first 10 elements


array([1., 2., 2., 1., 1., 2., 1., 1., 2., 2.])

In [22]:
with open('squats.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

#'squats.pkl' contains the serialized (pickled) 
# version of the trained Random Forest classifier (fit_models['rf'])

In [23]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [33]:
#video feed
cap = cv2.VideoCapture('squats.mp4') #this number is the number of my web cam

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
#videoWriter = cv2.VideoWriter('press2.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (int(width), int(height)))
current_stage = ''
counter = 0
stage = ''
feedback = 'he'

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        # detect stuff and render
        # print(landmarks)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False #save a bunch of memory

        #make detections from the pose
        results = pose.process(image) #by processing it, we get our detections back
    
        #back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        try: 
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]). flatten()
            X = pd.DataFrame([row], columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            #up is class 1; down is class 2

            if body_language_class == 1 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 2 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'up'
                counter += 1

            if body_language_class == 1:
                stage = "up"
            elif body_language_class == 2:
                stage = "down"

            # Drawing rectangles and text on the image
            cv2.rectangle(image, (0,0), (400,60), (245,117,16), -1)
            
            # Drawing predicted class
            cv2.putText(image, "CLASS", (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(stage), (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            # Drawing predicted probability
            cv2.putText(image, "PROB", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            # Drawing counter
            cv2.putText(image, "COUNTER", (180,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (175,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)

        except Exception as e:
            print(e)
            break

    
        cv2.imshow('Mediapipe Feed', image)        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99950852e-01 4.91477536e-05]
1.0 [9.99980602e-01 1.93983001e-05]
1.0 [9.99980602e-01 1.93983001e-05]
1.0 [9.99980602e-01 1.93983001e-05]
1.0 [9.99980602e-01 1.93983001e-05]
1.0 [9.99980602e-01 1.93983001e-05]
1.0 [9.99970277e-01 2.97233024e-05]
1.0 [9.99970277e-01 2.97233024e-05]
1.0 [9.99970277e-01 2.97233024e-05]
1.0 [9.99970277e-01 2.97233024e-05]
1.0 [9.99924694e-01 7.53061043e-05]
1.0 [9.99924694e-01 7.53061043e-05]
1.0 [9.99924694e-01 7.53061043e-05]
1.0 [9.99924694e-01 7.53061043e-05]
1.0 [9.99924694e-01 7.53061043e-05]
1.0 [9.99938465e-01 6.15354646e-05]
1.0 [9.99959840e-01 4.01602953e-05]
1.0 [9.99959840e-01 4.01602953e-05]
1.0 [9.99984149e-01 1.58509223e-05]
1.0 [9.99984149e-01 1.58509223e-05]
1.0 [9.99984149e-01 1.58509223e-05]
1.0 [9.99984149e-01 1.585092

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
